<a href="https://www.kaggle.com/code/blnikan/datafudsioncontest2022?scriptVersionId=96125665" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

блакнот на kaggle.com
https://www.kaggle.com/code/blnikan/datafudsioncontest2022

# Работа с данными для тренеровки

##### для начала посмотрим на те данные для которых мы уже знаем есть ли вышка

In [ ]:
train = pd.read_csv("../input/vtbdatafusion2022/train.csv")
print('train.csv - DONE!')

##### train.csv: Табличный файл с целевой переменной: меткой наличия у клиентов ВТБ высшего образования — 0 и 1. Разметка представлена не для всех клиентов

In [ ]:
train.head() #вывод верха таблицы

##### для начала найдем все транзакции для каждого из этих людей, а для этого мы загрузим еще несколько файлов. А так же соединим эти файлы, чтобы получить одну общую таблицы по всем этим людям

In [ ]:
tra_val = pd.read_csv('../input/vtbdatafusion2022/train_matching.csv')
print('train_matching.csv - DONE!')

##### train_matching.csv: Целевая переменная — табличный файл с точным соответствием между id клиентов в данных транзакций и кликстримов. Разметка представлена не для всех клиентов, присутствующих в данных транзакций и кликстрима

In [ ]:
# введем новую переменную в которую сведем первые 2 таблицы.
#стоит отметить что будем соединять по 1й табл, чтобы не потерять порядок людей!
df_H_edu_all = train.merge(tra_val)
df_H_edu_all.head()

##### далее загрузим файл в котором содержатся ВСЕ транзакции
( это займет какое-то время, тк файл не самый маленький )

In [ ]:
trans = pd.read_csv('../input/vtbdatafusion2022/transactions.csv')
print('transactions.csv - DONE!')

##### transactions.csv: Список транзакций

In [ ]:
trans.head()

теперь для людей из df_H_edu_all найдем все их траты и запишим их в отдельную таблицу, но для начала надо именить названия поля в таблице trans,  чтобы названия были одинаковыми.

In [ ]:
trans.rename(columns = {'user_id': 'bank'}, inplace = True)
trans

In [ ]:
#объединяем 
df_tran_H_edu_all = df_H_edu_all.merge(trans)
df_tran_H_edu_all.head()

в приличном обществе за то что вы увидите дальше я бы получил по лицу, Но пока что есть, то есть

In [ ]:
jss =  df_tran_H_edu_all.shape
jss = jss[0]
print(jss)

In [ ]:
mon_div_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_USD = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_USD = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_EUR = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_EUR = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_xz = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_xz = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

col_VO_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mcc_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
i = 0
j = 0


mon_div_all[i] += float(df_tran_H_edu_all.transaction_amt[j])
mcc[i] += int(df_tran_H_edu_all.mcc[j])


for j in  range (jss-1):
    #подсчет процентов, чтобы знать сколько будет делать
    if j/(jss//100) == int(j/(jss//100)): 
        print(j/(jss//100),'%')
        
    h1 = str(df_tran_H_edu_all.bank[j] )
    h2 = str(df_tran_H_edu_all.bank[j+1])
    
    if h1 == h2: # сверяю что идентифик j и j+1 одинаковые и тогда считаю для разных валют
        
        m2 = float(df_tran_H_edu_all.transaction_amt[j+1]) #деньги (j+1)-го
        
        col_VO_all [i] += 1 #кол-во транзакций для каждого человека
        
        temp_mcc = int(df_tran_H_edu_all.mcc[j+1])
        #в этом промежутке самое интересное, а именно траты на авиа белеты и прочее
        if int(df_tran_H_edu_all.mcc[j+1])>3000: 
            if int(df_tran_H_edu_all.mcc[j+1]) < 4000:
                temp_mcc *= 2
        mcc[i] += temp_mcc #считаю  mcc общий mcc код для каждого
            
        # тут определяю какая валюта и было списание или пополнение денег
        if float(df_tran_H_edu_all.currency_rk[j+1]) == float(48): 
            
            if m2 > 0:
                mon_plu_all[i] +=m2
            else:
                mon_div_all[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(50):
            
            if m2 > 0:
                mon_plu_all_USD[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(60):
            
            if m2 > 0:
                mon_plu_all_EUR[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        else:
            
            if m2 > 0:
                mon_plu_all_xz[i] +=m2
            else:
                mon_div_all_xz[i] += m2
            
    else:
        m2 = float(df_tran_H_edu_all.transaction_amt[j+1])
        
        if float(df_tran_H_edu_all.currency_rk[j+1]) == float(48): #
            
            if m2 > 0:
                mon_plu_all[i] +=m2
            else:
                mon_div_all[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(50):
            
            if m2 > 0:
                mon_plu_all_USD[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(60):
            
            if m2 > 0:
                mon_plu_all_EUR[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        else:
            
            if m2 > 0:
                mon_plu_all_xz[i] +=m2
            else:
                mon_div_all_xz[i] += m2
        i += 1


print(mon_div_all)
print(mon_plu_all)

print(mon_div_all_USD)
print(mon_plu_all_USD)
print(mon_div_all_EUR)
print(mon_plu_all_EUR)
print(mon_div_all_xz)
print(mon_plu_all_xz)
print()
print('col-VO\n',col_VO_all)

теперь добавляем эти данные в таблицу df_H_edu_all 
стоит заметить что именно для этого нам надо было не потерять порядок людей

In [ ]:
df_H_edu_all['mon_div_all'] = mon_div_all
df_H_edu_all['mon_plu_all'] = mon_plu_all

df_H_edu_all['mon_div_all_USD'] = mon_div_all_USD
df_H_edu_all['mon_plu_all_USD'] = mon_plu_all_USD

df_H_edu_all['mon_div_all_EUR'] = mon_div_all_EUR
df_H_edu_all['mon_plu_all_EUR'] = mon_plu_all_EUR

df_H_edu_all['mcc_all'] = mcc_all
df_H_edu_all['col_VO_all'] = col_VO_all

## Построение дерева

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X = df_H_edu_all[[ 'mon_div_all', 'mon_plu_all', 
                  'mon_div_all_USD', 'mon_plu_all_USD',
                  'mon_div_all_EUR','mon_plu_all_EUR',
                  'mcc_all' , 'col_VO_all'
                 ]]
y = df_H_edu_all['higher_education']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=85)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

теперь определим точность которую мы получаем

In [ ]:
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))